In [1]:
import dimod
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
from dwave.system import LeapHybridDQMSampler

In [2]:
# locations (locs) and facilities should be same-sized lists
locs = list('abcd')
facilities = ['hq', 'rnd', 'depo', 'factory']

In [3]:
# size of the problem to consider - useful to consider smaller problem to that solves quickly (cannot be larger than len(locs)!)
m = 4  

In [4]:
dist_bw_cities = np.array([[0, 10, 40, 30], [10, 0, 20, 50], [40, 20, 0, 20], [30, 50, 20, 0]])
dbc = pd.DataFrame(dist_bw_cities[:m, :m], index=locs[:m], columns=locs[:m])
print("Distance between cities:")
print(dist_bw_cities)

Distance between cities:
[[ 0 10 40 30]
 [10  0 20 50]
 [40 20  0 20]
 [30 50 20  0]]


In [5]:
flows_bw_facilities = np.array([[0, 30, 5, 10], [30, 0, 5, 20], [5, 5, 0, 50], [10, 20, 50, 0]])
fbf = pd.DataFrame(flows_bw_facilities[:m, :m], index=facilities[:m], columns=facilities[:m])
print("Flows between facilities:")
print(flows_bw_facilities)

Flows between facilities:
[[ 0 30  5 10]
 [30  0  5 20]
 [ 5  5  0 50]
 [10 20 50  0]]


In [6]:
n_city=4
n_facs=4

Objective function to be minimised - $flow*distance$ for chosen $x$
$$
H_{1} = min\sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{k=1}^{n}\sum_{l=1}^{n}(f_{ij}.d_{kl})x_{i,k}x_{j,l}
$$
with $x_{i,k}$ being a binary variable, which is $1$ if facility $i$ is assigned to location $k$ and $0$ otherwise. The same applies for the binary variable $x_{j,l}$

In [7]:
dqm = dimod.DiscreteQuadraticModel()

In [8]:
for j in range(n_facs):
    dqm.add_variable(n_city, label=f'facs[{j}]')

dqm.variables

Variables(['facs[0]', 'facs[1]', 'facs[2]', 'facs[3]'])

In [9]:
for i in range(n_facs):
    for j in range(n_facs):
        for m in range(n_city):
            dqm.set_linear_case(f'facs[{i}]',m,dqm.get_linear_case(f'facs[{i}]', m) + (dist_bw_cities[m][m] * flows_bw_facilities[i][i]))                  
            dqm.set_linear_case(f'facs[{j}]',m,dqm.get_linear_case(f'facs[{j}]', m) + (dist_bw_cities[m][m] * flows_bw_facilities[j][j]))
            for n in range(n_city):
                if i != j:
                    # print(f'city[{i}]', m, f'city[{j}]', n, dqm.get_quadratic_case(f'city[{i}]', m, f'city[{j}]',n),(flows_bw_facilities[m][n] * dist_bw_cities[i][j]))
                    dqm.set_quadratic_case(f'facs[{i}]', m, f'facs[{j}]', n, dqm.get_quadratic_case(f'facs[{i}]', m, f'facs[{j}]',n) + (dist_bw_cities[m][n] * flows_bw_facilities[i][j]))
                    

In [10]:
max_bound = np.max(dist_bw_cities) * np.max(flows_bw_facilities) + 1
# max_bound = 10000
print(f'max_bound: {max_bound}')

max_bound: 2501


In [11]:
for i in range(n_facs):
    for m in range(n_city):
        dqm.set_linear_case(f'facs[{i}]',m,dqm.get_linear_case(f'facs[{i}]', m) - 2*max_bound)                  
        for j in range(n_facs):
            for n in range(n_city):
                if i != j:
                    if m == n:
                        dqm.set_quadratic_case(f'facs[{i}]', m, f'facs[{j}]', n, dqm.get_quadratic_case(f'facs[{i}]', m, f'facs[{j}]',n) + max_bound)                        

In [12]:
# Load bqm model from earlier QAP implementation and then check biases against each other 

# for i in range(n_facs):
#     for j in range(n_facs):
#         for k in range(n_city):
#             for l in range(n_city):
#                 if i!=j:
#                     print(f'x[{i}][{k}]', f'x[{j}][{l}]', dqm.get_quadratic_case(f'facs[{i}]', k, f'facs[{j}]', l) == bqm.get_quadratic(f'x[{i}][{k}]', f'x[{j}][{l}]'))
#                 else:
#                     print(f'x[{i}][{k}]', dqm.get_linear_case(f'facs[{i}]', k) == bqm.get_linear(f'x[{i}][{k}]'))

In [13]:
ENDPOINT = 'https://cloud.dwavesys.com/sapi/'
TOKEN = 'DEV-d8e37a0cf77b13dea4e2ea7be2afe998b23e674a' # Enter token here

In [14]:
sampler = LeapHybridDQMSampler(token=TOKEN, endpoint=ENDPOINT)
sampleset = sampler.sample_dqm(dqm, label='DQM')

In [15]:
print(sampleset)

   facs[0] facs[1] facs[2] facs[3]   energy num_oc.
3        3       2       0       1 -15308.0       1
4        3       2       0       1 -15308.0       1
6        3       2       0       1 -15308.0       1
7        3       2       0       1 -15308.0       1
8        3       2       0       1 -15308.0       1
11       3       2       0       1 -15308.0       1
12       3       2       0       1 -15308.0       1
13       3       2       0       1 -15308.0       1
5        2       3       1       0 -15108.0       1
25       0       1       3       2 -15008.0       1
0        3       2       1       0 -14908.0       1
1        3       2       1       0 -14908.0       1
10       3       2       1       0 -14908.0       1
26       3       2       1       0 -14908.0       1
2        2       3       0       1 -14708.0       1
9        0       1       2       3 -14208.0       1
24       0       3       1       2 -14008.0       1
14       0       3       2       1 -13408.0       1
15       1  

In [17]:
sample = sampleset.first.sample
energy = sampleset.first.energy
sample, energy

({'facs[0]': 3, 'facs[1]': 2, 'facs[2]': 0, 'facs[3]': 1}, -15308.0)

In [16]:
# # Naive approach

# t_start = datetime.now()
# m=4
# # Create permutation matrices and find the total cost i.e. flow*distance, associated with the matrix
# permutation_mapping = {}
# for i, row in enumerate(np.eye(m)):
#     permutation_mapping[i+1] = row

# cost_matrix_mapping = {}
# for p in list(permutations(range(1, m+1))):
#     permutation_matrix = np.array([permutation_mapping[x] for x in p]).reshape(m, m)
#     cost = np.sum(flows_bw_facilities * (permutation_matrix @ dist_bw_cities @ permutation_matrix.T))
#     cost_matrix_mapping[p] = cost

# # Find the permutation matrix with the lowest cost 

# opt_permutation = min(cost_matrix_mapping, key=cost_matrix_mapping.get)
# opt = np.array([permutation_mapping[x] for x in opt_permutation]).reshape(m, m)

# end_time = datetime.now()

# # Optimal solution as a 2D matrix
# print(opt)
# print(f'optimal solution (minimum of flow*dist possible): {min(cost_matrix_mapping.values())}')


# print(f'\ntime taken for finding matrix with optimal costs: {end_time - t_start}')